In [2]:

from datasets import load_dataset, DatasetDict, load_from_disk
preprocessed_splits = load_from_disk("../wikitext-103-preprocessed-ws-notext-gpt2-128-wtest-v2")


In [3]:
print(len(preprocessed_splits["train"]))

1014634


In [7]:
for i in range(10):
    print("preprocessed_splits train : ", preprocessed_splits["train"][i])

preprocessed_splits train :  {'input_ids': [20665, 14064, 710, 21566, 22863, 837, 257, 3830, 32803, 283, 286, 262, 2908, 29350, 70, 341, 390, 15085, 79, 385, 837, 11406, 319, 20111, 5451, 319, 3269, 362, 837, 1248, 2414, 837, 319, 262, 1987, 400, 1110, 286, 465, 12928, 422, 3254, 1845, 64, 8836, 568, 764, 679, 373, 284, 3520, 319, 20111, 5451, 329, 5193, 1933, 837, 17138, 2890, 663, 17622, 764, 679, 2630, 281, 1848, 286, 465, 2652, 287, 543, 339, 3136, 465, 9412, 286, 262, 17255, 326, 614, 1058, 220, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
print(preprocessed_splits["test"][0].keys())

dict_keys(['overflowing_tokens', 'input_ids', 'attention_mask'])


In [1]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") # 论文有说是increase 到1024
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 128

In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, GPT2Tokenizer, GPT2Config
tokenizer2 = AutoTokenizer.from_pretrained("./tokenizer_save/tokenizer-gpt2-128")

In [ ]:
print(tokenizer.name_or_path)

In [2]:
tokenizer.save_pretrained(f"./tokenizer_save/tokenizer-{tokenizer.name_or_path}-{tokenizer.model_max_length}")

('./tokenizer_save/tokenizer-gpt2-128/tokenizer_config.json',
 './tokenizer_save/tokenizer-gpt2-128/special_tokens_map.json',
 './tokenizer_save/tokenizer-gpt2-128/vocab.json',
 './tokenizer_save/tokenizer-gpt2-128/merges.txt',
 './tokenizer_save/tokenizer-gpt2-128/added_tokens.json')

In [6]:
preprocessed_splits["test"] = load_dataset("wikitext", "wikitext-103-raw-v1", split="test")

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [7]:
preprocessed_splits["test"].save_to_disk("wikitext-103-raw-test")

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

In [7]:
print(len(preprocessed_splits["test"]))

0


In [ ]:
for i in range(10):
    print(preprocessed_splits["test"][i])

In [8]:
print(len(preprocessed_splits["test"]))
print(type(preprocessed_splits["test"][0]["overflowing_tokens"]))

4358
<class 'list'>


### 下面是一套文本类test的数据集预处理组合
主要实现文本转token，batch长度，test的标准答案overflowing token转化成ID，并且要能有比较长的

In [9]:
# 这里存在的问题就是overflowing_tokens 长短不一，没办法batch
def preprocess_function(example):
    return_dic = tokenizer.__call__(example["text"], padding="max_length", truncation=True,   # 在这就不带文本了
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True) 

    # if len(return_dic["overflowing_tokens"]) > 0:
    #     return_dic["overflow_text"] = tokenizer.decode(return_dic["overflowing_tokens"])
    # else:
    #     return_dic["overflow_text"] = ""
    # return_dic["prompt"] = tokenizer.decode(return_dic["input_ids"])
    return return_dic
#Apply preprocessing to dataset
preprocessed_splits["test"] = preprocessed_splits["test"].map(preprocess_function,batched=True, remove_columns=["text"],num_proc=4)



Loading cached processed dataset at /home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-ba496d90cb8b690f_*_of_00004.arrow


In [10]:
def filter_function(example):  # 就没有大于300的
    return len(example['overflowing_tokens']) > 20 and len(example['overflowing_tokens']) < 300 

preprocessed_splits["test"] = preprocessed_splits["test"].filter(filter_function,num_proc=4)

Loading cached processed dataset at /home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-99423aefded79e6e_*_of_00004.arrow


In [13]:
print(len(preprocessed_splits["test"]))

757


In [ ]:
for i in range(10):
    print(preprocessed_splits["test"][i])

In [14]:
def preprocess_function_truncate(example, max_over_length=128):
    # if isinstance(example["overflowing_tokens"], list):
    #     example["overflowing_tokens"] = example["overflowing_tokens"].truncate(max_over_length)
    # else:
    # print("type: ",type(example["overflowing_tokens"]))
    def complete_list(lst, length, value=tokenizer.eos_token_id):
        l = len(lst)
        return lst + [value] * (length - l)
    if len(example["overflowing_tokens"]) > max_over_length:
        example["overflowing_tokens"] = example["overflowing_tokens"][:max_over_length]
    else:
        example["overflowing_tokens"] = complete_list(example["overflowing_tokens"], max_over_length)
    return example

preprocessed_splits["test"] = preprocessed_splits["test"].map(preprocess_function_truncate, batched=False, num_proc=4)

Map (num_proc=4):   0%|          | 0/757 [00:00<?, ? examples/s]

In [18]:
try:
    preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns(["overflowing_tokens_ids"])
except:
    pass
try:
    preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns(["num_truncated_tokens"])
except:
    pass

In [ ]:
# 最开始没看懂这个batch，用下面的函数验证
# def preprocess_function_truncate(example, max_over_length=128):
#     print("type: ",type(example["overflowing_tokens"]))
#     print("length: ",len(example["overflowing_tokens"]))
#     print("type: ",type(example["overflowing_tokens"][0]))
#     print("length: ",len(example["overflowing_tokens"][0]))
#     return example
# preprocessed_splits["test"] = preprocessed_splits["test"].map(preprocess_function_truncate, batched=True)

In [5]:

# def complete_list(lst, length, value=tokenizer.eos_token_id):
#     """
#     补全list到指定长度
#     :param lst: 原list
#     :param length: 指定长度
#     :param value: 补全的值，默认为None
#     :return: 补全后的list
#     """
#     l = len(lst)
#     return lst + [value] * (length - l)

# for i in range(len(preprocessed_splits["test"])):
#     if(len(preprocessed_splits["test"][i]["overflowing_tokens"]) >= 128):
#         preprocessed_splits["test"][i]["overflowing_tokens"] = preprocessed_splits["test"][i]["overflowing_tokens"][:128]
#     else:
#         preprocessed_splits["test"][i]["overflowing_tokens"] = complete_list(preprocessed_splits["test"][i]["overflowing_tokens"], 128)
    

In [ ]:
for i in range(10):
    print(preprocessed_splits["test"][i])

In [ ]:
preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns("num_truncated_tokens")


In [3]:
preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns("overflowing_tokens_truncate")

In [20]:
preprocessed_splits.save_to_disk("wikitext-103-preprocessed-ws-notext-gpt2-128-wtest-v2")#wikitext-103-preprocessed-ws-notext-gpt2-128

Saving the dataset (0/2 shards):   0%|          | 0/1014634 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/112738 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/757 [00:00<?, ? examples/s]